# Build Docker image 

In this notebook, we will build the docker container that contains the ResNet152 model, Flask web application, model driver and all dependencies.
Make sure you have logged in using docker login.

In [1]:
import os
from os import path
import json
import shutil
from dotenv import set_key, get_key

We will be using the following Docker information to push the image to docker hub.

In [2]:
# "YOUR_DOCKER_LOGIN"

In [3]:
%%writefile .env
# This cell is tagged `parameters`
# Please modify the values below as you see fit

# Your docker login and image repository name



Overwriting .env


In [4]:
set_key('.env', 'docker_login', 'masalvar')

(True, 'docker_login', 'masalvar')

In [5]:
set_key('.env', 'image_repo', 'pytorch-gpu')

(True, 'image_repo', 'pytorch-gpu')

In [6]:
!cat .env

# This cell is tagged `parameters`
# Please modify the values below as you see fit

# Your docker login and image repository name
docker_login="masalvar"
image_repo="pytorch-gpu"


In [7]:
os.makedirs('flaskwebapp', exist_ok=True)
os.makedirs(os.path.join('flaskwebapp', 'nginx'), exist_ok=True)
os.makedirs(os.path.join('flaskwebapp', 'etc'), exist_ok=True)

In [8]:
shutil.copy('synset.txt', 'flaskwebapp')
shutil.copy('driver.py', 'flaskwebapp')
os.listdir('flaskwebapp')

['synset.txt', 'etc', 'driver.py', 'nginx']

Below, we create the module for the Flask web application.

In [9]:
%%writefile flaskwebapp/app.py

from flask import Flask, request
import logging
import json
import driver

app = Flask(__name__)
predict_for = driver.get_model_api()
 
@app.route("/score", methods = ['POST'])
def scoreRRS():
    """ Endpoint for scoring
    """
    if request.headers['Content-Type'] != 'application/json':
        return Response(json.dumps({}), status= 415, mimetype ='application/json')
    request_input = request.json['input']
    response = predict_for(request_input)
    return json.dumps({'result': response})


@app.route("/")
def healthy():
    return "Healthy"

# Tensorflow Version
@app.route('/version', methods = ['GET'])
def version_request():
    return driver.version()

if __name__ == "__main__":
    app.run(host='0.0.0.0', port=5000)

Writing flaskwebapp/app.py


In [10]:
%%writefile flaskwebapp/wsgi.py
from app import app as application

def create():
    print("Initialising")
    application.run(host='127.0.0.1', port=5000)

Writing flaskwebapp/wsgi.py


Here, we write the configuration for the Nginx which creates a proxy between ports **80** and **5000**.

In [11]:
%%writefile flaskwebapp/nginx/app
server {
    listen 80;
    server_name _;
 
    location / {
    include proxy_params;
    proxy_pass http://127.0.0.1:5000;
    proxy_connect_timeout 5000s;
    proxy_read_timeout 5000s;
  }
}

Writing flaskwebapp/nginx/app


In [12]:
%%writefile flaskwebapp/gunicorn_logging.conf

[loggers]
keys=root, gunicorn.error

[handlers]
keys=console

[formatters]
keys=json

[logger_root]
level=INFO
handlers=console

[logger_gunicorn.error]
level=ERROR
handlers=console
propagate=0
qualname=gunicorn.error

[handler_console]
class=StreamHandler
formatter=json
args=(sys.stdout, )

[formatter_json]
class=jsonlogging.JSONFormatter

Writing flaskwebapp/gunicorn_logging.conf


In [13]:
%%writefile flaskwebapp/kill_supervisor.py
import sys
import os
import signal

def write_stdout(s):
    sys.stdout.write(s)
    sys.stdout.flush()

# this function is modified from the code and knowledge found here: http://supervisord.org/events.html#example-event-listener-implementation
def main():
    while 1:
        write_stdout('READY\n')
        # wait for the event on stdin that supervisord will send
        line = sys.stdin.readline()
        write_stdout('Killing supervisor with this event: ' + line);
        try:
            # supervisord writes its pid to its file from which we read it here, see supervisord.conf
            pidfile = open('/tmp/supervisord.pid','r')
            pid = int(pidfile.readline());
            os.kill(pid, signal.SIGQUIT)
        except Exception as e:
            write_stdout('Could not kill supervisor: ' + e.strerror + '\n')
            write_stdout('RESULT 2\nOK')

main()

Writing flaskwebapp/kill_supervisor.py


In [14]:
%%writefile flaskwebapp/etc/supervisord.conf 
[supervisord]
logfile=/tmp/supervisord.log ; (main log file;default $CWD/supervisord.log)
logfile_maxbytes=50MB        ; (max main logfile bytes b4 rotation;default 50MB)
logfile_backups=10           ; (num of main logfile rotation backups;default 10)
loglevel=info                ; (log level;default info; others: debug,warn,trace)
pidfile=/tmp/supervisord.pid ; (supervisord pidfile;default supervisord.pid)
nodaemon=true                ; (start in foreground if true;default false)
minfds=1024                  ; (min. avail startup file descriptors;default 1024)
minprocs=200                 ; (min. avail process descriptors;default 200)

[program:gunicorn]
command=bash -c "gunicorn --workers 1 -m 007 --timeout 100000 --capture-output --error-logfile - --log-level debug --log-config gunicorn_logging.conf \"wsgi:create()\""
directory=/code
redirect_stderr=true
stdout_logfile =/dev/stdout
stdout_logfile_maxbytes=0
startretries=2
startsecs=20

[program:nginx]
command=/usr/sbin/nginx -g "daemon off;"
startretries=2
startsecs=5
priority=3

[eventlistener:program_exit]
command=python kill_supervisor.py
directory=/code
events=PROCESS_STATE_FATAL
priority=2

Writing flaskwebapp/etc/supervisord.conf


We create a custom image based on the CUDA 9 image from NVIDIA and install all the necessary dependencies. This is in order to try and keep the size of the image as small as possible.

In [15]:
%%writefile flaskwebapp/requirements.txt
Pillow==5.0.0
click==6.7
configparser==3.5.0
Flask==0.12.2
gunicorn==19.6.0
json-logging-py==0.2
MarkupSafe==1.0
olefile==0.44
requests==2.12.3

Writing flaskwebapp/requirements.txt


In [16]:
%%writefile flaskwebapp/dockerfile

FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04

RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list

RUN mkdir /code
WORKDIR /code
ADD . /code/
ADD etc /etc

RUN apt-get update && apt-get install -y --no-install-recommends \
        build-essential \
        ca-certificates \
        cmake \
        curl \
        git \
        nginx \
        supervisor \
        wget && \
        rm -rf /var/lib/apt/lists/*


ENV PYTHON_VERSION=3.6
RUN curl -o ~/miniconda.sh -O  https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh  && \
    chmod +x ~/miniconda.sh && \
    ~/miniconda.sh -b -p /opt/conda && \
    rm ~/miniconda.sh && \
    /opt/conda/bin/conda create -y --name py$PYTHON_VERSION python=$PYTHON_VERSION numpy scipy pandas scikit-learn && \
    /opt/conda/bin/conda clean -ya
ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
ENV PYTHONPATH /code/:$PYTHONPATH

    
RUN rm /etc/nginx/sites-enabled/default && \
    cp /code/nginx/app /etc/nginx/sites-available/ && \
    ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ && \
    /opt/conda/bin/conda install -c pytorch pytorch==0.4.1 && \
    pip install --upgrade pip && \
    pip install torchvision==0.2.1 && \
    pip install -r /code/requirements.txt && \       
    /opt/conda/bin/conda clean -yt

EXPOSE 80
CMD ["supervisord", "-c", "/code/etc/supervisord.conf"]

Writing flaskwebapp/dockerfile


The image name below refers to our dockerhub account. If you wish to push the image to your account make sure you change the docker login.

In [17]:
image_name = get_key('.env', 'docker_login') + '/' +get_key('.env', 'image_repo') 
application_path = 'flaskwebapp'
docker_file_location = path.join(application_path, 'dockerfile')

Next, we build our docker image. The output of this cell is cleared from this notebook as it is quite long due to all the installations required to build the image. However, you should make sure you see 'Successfully built' and 'Successfully tagged' messages in the last line of the output when you run the cell. 

In [18]:
!docker build -t $image_name -f $docker_file_location $application_path --no-cache

Sending build context to Docker daemon  50.18kB
Step 1/15 : FROM nvidia/cuda:9.0-cudnn7-devel-ubuntu16.04
 ---> 33d7c76ce726
Step 2/15 : RUN echo "deb http://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1604/x86_64 /" > /etc/apt/sources.list.d/nvidia-ml.list
 ---> Running in b26b9ce14118
Removing intermediate container b26b9ce14118
 ---> 867e076ded01
Step 3/15 : RUN mkdir /code
 ---> Running in de1ff3992580
Removing intermediate container de1ff3992580
 ---> 1ffdbe445806
Step 4/15 : WORKDIR /code
Removing intermediate container c92b1813e275
 ---> 85e01db99785
Step 5/15 : ADD . /code/
 ---> 5e8954bc1887
Step 6/15 : ADD etc /etc
 ---> 24a109c5b58e
Step 7/15 : RUN apt-get update && apt-get install -y --no-install-recommends         build-essential         ca-certificates         cmake         curl         git         nginx         supervisor         wget &&         rm -rf /var/lib/apt/lists/*
 ---> Running in 59536f8a40c2
Get:1 http://archive.ubuntu.com/ubuntu xenial 

Get:25 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxdmcp6 amd64 1:1.1.2-1.1 [11.0 kB]
Get:26 http://archive.ubuntu.com/ubuntu xenial/main amd64 libxcb1 amd64 1.11.1-1ubuntu1 [40.0 kB]
Get:27 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libx11-data all 2:1.6.3-1ubuntu2.1 [113 kB]
Get:28 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 libx11-6 amd64 2:1.6.3-1ubuntu2.1 [570 kB]
Get:29 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 wget amd64 1.17.1-1ubuntu1.4 [299 kB]
Get:30 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 curl amd64 7.47.0-1ubuntu2.9 [138 kB]
Get:31 http://archive.ubuntu.com/ubuntu xenial/main amd64 fonts-dejavu-core all 2.35-1 [1039 kB]
Get:32 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 fontconfig-config all 2.11.94-0ubuntu1.1 [49.9 kB]
Get:33 http://archive.ubuntu.com/ubuntu xenial/main amd64 liberror-perl all 0.17-1.2 [19.6 kB]
Get:34 http://archive.ubuntu.com/ubuntu xenial-updates/main amd64 git-

Selecting previously unselected package fonts-dejavu-core.
Preparing to unpack .../fonts-dejavu-core_2.35-1_all.deb ...
Unpacking fonts-dejavu-core (2.35-1) ...
Selecting previously unselected package fontconfig-config.
Preparing to unpack .../fontconfig-config_2.11.94-0ubuntu1.1_all.deb ...
Unpacking fontconfig-config (2.11.94-0ubuntu1.1) ...
Selecting previously unselected package liberror-perl.
Preparing to unpack .../liberror-perl_0.17-1.2_all.deb ...
Unpacking liberror-perl (0.17-1.2) ...
Selecting previously unselected package git-man.
Preparing to unpack .../git-man_1%3a2.7.4-0ubuntu1.4_all.deb ...
Unpacking git-man (1:2.7.4-0ubuntu1.4) ...
Selecting previously unselected package git.
Preparing to unpack .../git_1%3a2.7.4-0ubuntu1.4_amd64.deb ...
Unpacking git (1:2.7.4-0ubuntu1.4) ...
Selecting previously unselected package libfreetype6:amd64.
Preparing to unpack .../libfreetype6_2.6.1-0.1ubuntu2.3_amd64.deb ...
Unpacking libfreetype6:amd64 (2.6.1-0.1ubuntu2.3) ...
Selecting pre

installing: pyopenssl-18.0.0-py37_0 ...
installing: urllib3-1.23-py37_0 ...
installing: requests-2.19.1-py37_0 ...
installing: conda-4.5.11-py37_0 ...
installation finished.
Solving environment: ...working... done
mkl-2019.0           | 204.4 MB  | ########## | 100% 
pandas-0.23.4        | 10.1 MB   | ########## | 100% 
intel-openmp-2019.0  | 721 KB    | ########## | 100% 
six-1.11.0           | 21 KB     | ########## | 100% 
scikit-learn-0.20.0  | 5.8 MB    | ########## | 100% 
scipy-1.1.0          | 18.0 MB   | ########## | 100% 
blas-1.0             | 6 KB      | ########## | 100% 
pip-10.0.1           | 1.8 MB    | ########## | 100% 
python-3.6.6         | 29.4 MB   | ########## | 100% 
certifi-2018.8.24    | 140 KB    | ########## | 100% 
numpy-base-1.15.2    | 4.2 MB    | ########## | 100% 
pytz-2018.5          | 232 KB    | ########## | 100% 
python-dateutil-2.7. | 260 KB    | ########## | 100% 
libgfortran-ng-7.3.0 | 1.3 MB    | ########## | 100% 
setuptools-40.4.3    | 555 KB 

Removing intermediate container 60e85319bb53
 ---> 6c10cb64bfc4
Step 10/15 : ENV PATH /opt/conda/envs/py$PYTHON_VERSION/bin:$PATH
 ---> Running in c6476494b1ed
Removing intermediate container c6476494b1ed
 ---> d7b1fa3bef39
Step 11/15 : ENV LD_LIBRARY_PATH /opt/conda/envs/py$PYTHON_VERSION/lib:/usr/local/cuda/lib64/:$LD_LIBRARY_PATH
 ---> Running in 0a9783b3dbe8
Removing intermediate container 0a9783b3dbe8
 ---> f6db8dc9bb8a
Step 12/15 : ENV PYTHONPATH /code/:$PYTHONPATH
 ---> Running in 0b54fa9fab43
Removing intermediate container 0b54fa9fab43
 ---> 05e316a54bde
Step 13/15 : RUN rm /etc/nginx/sites-enabled/default &&     cp /code/nginx/app /etc/nginx/sites-available/ &&     ln -s /etc/nginx/sites-available/app /etc/nginx/sites-enabled/ &&     /opt/conda/bin/conda install -c pytorch pytorch==0.4.1 &&     pip install --upgrade pip &&     pip install torchvision==0.2.1 &&     pip install -r /code/requirements.txt &&     /opt/conda/bin/conda clean -yt
 ---> Running in 519efd0e1a81
Solving

  Running setup.py bdist_wheel for configparser: started
  Running setup.py bdist_wheel for configparser: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/a3/61/79/424ef897a2f3b14684a7de5d89e8600b460b89663e6ce9d17c
  Running setup.py bdist_wheel for json-logging-py: started
  Running setup.py bdist_wheel for json-logging-py: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/0d/2e/1c/c638b7589610d8b9358a6e5eb008edacb8b3e9b6d1edc9479f
  Running setup.py bdist_wheel for MarkupSafe: started
  Running setup.py bdist_wheel for MarkupSafe: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/33/56/20/ebe49a5c612fffe1c5a632146b16596f9e64676768661e4e46
  Running setup.py bdist_wheel for olefile: started
  Running setup.py bdist_wheel for olefile: finished with status 'done'
  Stored in directory: /root/.cache/pip/wheels/c4/19/76/61fc7929d808e51567aff23036ca5fe6ba8336ad0559ca6a27
  Running setup.py bdist_wheel for itsdanger

Below we will push the image created to our dockerhub registry. Make sure you have already logged in to the appropriate dockerhub account using the docker login command. If you haven't loged in to the approrpiate dockerhub account you will get an error.

In [19]:
!docker push $image_name

The push refers to repository [docker.io/masalvar/pytorch-gpu]

8bf6c682: Preparing 
9bbe35c0: Preparing 
607bbf34: Preparing 
e8b36243: Preparing 
27831895: Preparing 
8e1f55ed: Preparing 
ed816181: Preparing 
96adce46: Preparing 
feb82a15: Preparing 
4e80c467: Preparing 
e6a06cff: Preparing 
6003d9a3: Preparing 
e6bcf901: Preparing 
ed816181: Waiting g 
b2f378bb: Preparing 
96adce46: Waiting g 
43c86cbc: Preparing 


bbe35c0: Pushing  1.028GB/1.279GBPushing  388.2MB/3.415GB

bf6c682: Pushing  2.736GB/3.415GBPushing  1.511GB/3.415GBPushing  1.836GB/3.415GB

bf6c682: Pushed   3.423GB/3.415GBPushing  3.198GB/3.415GBlatest: digest: sha256:288c5f9940eedb839315b62b8fce722fb9760c1ca522c1411359a99ff02e4164 size: 4093


In [20]:
print('Docker image name {}'.format(image_name))

Docker image name masalvar/pytorch-gpu


We can now [test our image locally](03_TestLocally.ipynb).